In [1]:
import os
os.environ["OPENCV_LOG_LEVEL"]="SILENT"

from pynq.overlays.base import BaseOverlay
from pynq.mmio import MMIO
from pynq.lib.video import * 

import PIL.Image
import cv2
import numpy as np

KeyboardInterrupt: 

In [ ]:
#   Copyright (c) 2017, Xilinx, Inc.
#   SPDX-License-Identifier: BSD-3-Clause


import pynq
import pynq.lib
import pynq.lib.video

class MyOverlay(pynq.Overlay):
    """ The Base overlay for the Pynq-Z2

    This overlay is designed to interact with all of the on board peripherals
    and external interfaces of the Pynq-Z2 board. It exposes the following
    attributes:

    Attributes
    ----------
    video : pynq.lib.video.HDMIWrapper
         HDMI input and output interfaces

    """

    def __init__(self, bitfile, **kwargs):
        super().__init__(bitfile, **kwargs)
        if self.is_loaded():
            pass



In [ ]:
# Loading of the base Overlay
#base = BaseOverlay("base.bit")

base = MyOverlay("/home/xilinx/work/test.bit")

In [ ]:
# Get the hdmi IN/OUT Modules
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

In [ ]:
help(hdmi_in)

In [ ]:
# Configure HDMI IN/OUT
# Input is configure to give gray scale

#test = VideoMode(1280,720, 24)

hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)

In [ ]:
# Start HDMI IN/OUT
hdmi_in.start()
hdmi_out.start()

In [ ]:
hdmi_in.tie(hdmi_out)

In [ ]:
frame = hdmi_in.readframe()
img = PIL.Image.fromarray(frame)
img.save("opencv_filters.jpg")

img

In [ ]:
#help(base.image_processing_pipeline.video_out_generator_0)
video_out_gen = MMIO(0x40010000, length=4096)

In [ ]:

#video_out_gen.write(0x0, 1)
#video_out_gen.write(0x4, 2)
#video_out_gen.write(0x8, 3)
#video_out_gen.write(0xc, 4)
#video_out_gen.write(0x10, 5)


print(video_out_gen.read(0x0))
print(video_out_gen.read(0x4))
print(video_out_gen.read(0x8))
print(video_out_gen.read(0xc))

In [ ]:
video_out_gen.write(0x4, 1)

In [ ]:
print("Horizontal Sync")
offset = 0x002c
print(base.video.hdmi_in.frontend.vtc_in.read(offset) & 0x00001FFF)
print((base.video.hdmi_in.frontend.vtc_in.read(offset) & 0xFFFF0000) >> 16)

print("\n")
offset += 4
print(base.video.hdmi_in.frontend.vtc_in.read(offset) & 0x00001FFF)
print((base.video.hdmi_in.frontend.vtc_in.read(offset) & 0xFFFF0000) >> 16)

print("\n")
offset += 4
print(base.video.hdmi_in.frontend.vtc_in.read(offset) & 0x00001FFF)
print((base.video.hdmi_in.frontend.vtc_in.read(offset) & 0xFFFF0000) >> 16)

print("\n")
print("\n")
print("Vertical Sync")
offset += 4
print(base.video.hdmi_in.frontend.vtc_in.read(offset) & 0x00001FFF)
print((base.video.hdmi_in.frontend.vtc_in.read(offset) & 0xFFFF0000) >> 16)

print("\n")
offset += 4
print(base.video.hdmi_in.frontend.vtc_in.read(offset) & 0x00001FFF)
print((base.video.hdmi_in.frontend.vtc_in.read(offset) & 0xFFFF0000) >> 16)

print("\n")
offset += 4
print(base.video.hdmi_in.frontend.vtc_in.read(offset) & 0x00001FFF)
print((base.video.hdmi_in.frontend.vtc_in.read(offset) & 0xFFFF0000) >> 16)

In [ ]:
# Gaussion approximation. Could easily be implemented as shifts
kernel_smoothing_values = np.array(
    [ 
        [ 1,  4,  6,  4, 1],
        [ 4, 16, 24, 16, 4],
        [ 6, 24, 36, 24, 6],
        [ 4, 16, 24, 16, 4],
        [ 1,  4,  6,  4, 1],
    ])
kernel_smoothing = (1/256) * kernel_smoothing_values 

print (kernel_smoothing)

# Smoothes frame_in
def image_smoothing(frame_in):
    """
        Smoothes frame_in with a 5x5 Gaussion Approximation Filter
    """
    result = cv2.filter2D(frame_in, cv2.CV_8U, kernel_smoothing)
    result = cv2.boxFilter(frame_in, cv2.CV_8U, [3, 3])
    display(PIL.Image.fromarray(result))
    return result
    

In [ ]:
# derivative Filter Kernels
kernel_edge_x = np.array([[1, 0, -1]])
kernel_edge_y = np.transpose(kernel_edge_x)

#print(kernel_edge_x)
#print(kernel_edge_y)


def edge_detection(frame_in):
    """
        Calculates the gradient in x and y direction with a normal derivative filter Kernel. Both
        the x and y derivative are returned.
    """
    gradient_x = cv2.filter2D(frame_in, cv2.CV_16S, kernel_edge_x)
    gradient_y = cv2.filter2D(frame_in, cv2.CV_16S, kernel_edge_y)
    
    #display(PIL.Image.fromarray(np.abs(gradient_x)))
    #display(PIL.Image.fromarray(np.abs(gradient_y)))
    
    return (gradient_x, gradient_y)
    

In [ ]:
def canny(gradient_x, gradient_y):
    """
        Takes the x and y gradient and calculates "true" edges. Uses canny edge detector.
    """

    threshold1 = 1
    threshold2 = 20
    
    # Compute the magnitude and direction of the gradient
    magnitude = np.hypot(gradient_x, gradient_y)
    #magnitude = np.abs(gradient_x) + np.abs(gradient_y)
    direction = np.arctan2(gradient_y, gradient_x) * 180/np.pi

    display(PIL.Image.fromarray(magnitude.astype(np.uint8)))
    
    # Perform non-maximum suppression to thin the edges
    
    
    M, N = magnitude.shape
    thin_edges = np.zeros((M, N), dtype=np.uint8)
    edge_rows, edge_cols = np.where(magnitude > 0)
    
    norm = 255 / np.max(magnitude)
    
    print(M,N, len(edge_rows), norm, np.max(magnitude))
    for index in range(1, len(edge_rows) - 2):
        #print("\r", i, end='')
       
        try:
            i = edge_rows[index]
            j = edge_cols[index]

            q = 255
            r = 255

           #angle 0
            if (0 <= np.abs(direction[i,j]) < 22.5) or (157.5 <= np.abs(direction[i,j]) <= 180):
                q = magnitude[i, j+1]
                r = magnitude[i, j-1]
            #angle 45
            elif (22.5 <= np.abs(direction[i,j]) < 67.5):
                q = magnitude[i+1, j-1]
                r = magnitude[i-1, j+1]
            #angle 90
            elif (67.5 <= np.abs(direction[i,j]) < 112.5):
                q = magnitude[i+1, j]
                r = magnitude[i-1, j]
            #angle 135
            elif (112.5 <= np.abs(direction[i,j]) < 157.5):
                q = magnitude[i-1, j-1]
                r = magnitude[i+1, j+1]

            if (magnitude[i,j] >= q) and (magnitude[i,j] >= r):
                thin_edges[i,j] = magnitude[i,j] * norm
            else:
                thin_edges[i,j] = 0
        except IndexError as e:
            pass
            
    
    print("Thin Edges")
    display(PIL.Image.fromarray(thin_edges))
    
    print("Hysteresis")
    # Perform hysteresis thresholding to segment the edges
    weak = 75
    strong = 255
    strong_rows, strong_cols = np.where(thin_edges >= threshold2)
    weak_rows, weak_cols = np.where((thin_edges <= threshold2) & (thin_edges >= threshold1))

    # Set strong edges to white
    edges = np.zeros_like(thin_edges)
    edges[strong_rows, strong_cols] = strong
    
    # Check for weak edges connected to strong edges
    for i in range(len(weak_rows)):
        row = weak_rows[i]
        col = weak_cols[i]
        if (edges[row - 1:row + 2, col - 1:col + 2] == strong).any():
            edges[row, col] = strong

    result = edges        
    
    display(PIL.Image.fromarray(result))
    
    return result

In [ ]:
def hough_transform(frame):
    """
        Calculates the hough trasform.
        Currently not fully implemented as the HoughCircles function really slow is. 
    """
    
    # Todo Implement
    
    result = cv2.HoughCircles(frame, cv2.HOUGH_GRADIENT, 1, 1)
    
    #print(result)
    #display(PIL.Image.fromarray(result))
    
    for i in range(0, len(result[0])):
        center = (int(result[0][i][0]), int(result[0][i][1]))
        cv2.circle(frame, center, int(result[0][i][2]), (255, 0, 255))
    
    display(PIL.Image.fromarray(frame))
    
    return result

In [ ]:
def detection(hough_space):
    """
        Detect circles in the HoughSpace.
        Not Implemented
    """
    # Todo Implement
    pass

In [ ]:
#
# Reading a frame and than handing it to the processing functions.
# Partial Images are displayed during processing from within the functions.
#

frame =  hdmi_in.readframe()

width, height = frame.shape
print(width, height)
small_frame = frame[540:1080, 400:960]
display(PIL.Image.fromarray(small_frame))

smooth = image_smoothing(small_frame)
gradients = edge_detection(smooth)
edges = canny(gradients[0], gradients[1])
hough = hough_transform(frame);

# Pass Through Setup

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import * 

In [ ]:
base = BaseOverlay("base.bit")

In [ ]:
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

In [ ]:

hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

In [ ]:

hdmi_in.start()
hdmi_out.start()

In [ ]:
hdmi_in.tie(hdmi_out);

In [ ]:
hdmi_in.stop()
hdmi_out.stop()

In [ ]:
help(VideoMode)